In [1]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['local']

subjects_collection = db['collectrium_stokstad_models_subjects']
makers = []
ids = []
n = 0

for artist in subjects_collection.find({'_type': 'Collectrium::Stokstad::Models::Artist', 'terms.0': {'$exists': True}}):
    
    if artist['approved']:
        for x in artist['terms']:
            makers.append(x['text'])
            ids.append(x['_id'])   

In [2]:
class TireNode:
    
    def __init__ (self):
        self.val = None
        self.mark = False
        self.index = None
        self.pointers = {}
        
class Tire:
    
    def __init__ (self):
        self.root = TireNode()
        
    def insert(self, name, index):
        self.re_insert(name, index, self.root)
        return
    
    def re_insert(self, name, index, root):
        if name[0] not in root.pointers:
            newNode = TireNode()
            newNode.val = name[0]
            root.pointers[name[0]] = newNode
            self.re_insert(name, index, root)
        else:
            nextNode = root.pointers[name[0]]
            if(len(name[1:]) == 0):
                nextNode.mark = True
                nextNode.index = index
                return
            return self.re_insert(name[1:], index, nextNode)
        
    def search(self, name):
        if len(name) == 0:
            return False
        else:
            return self.re_search(name, self.root)
        
    def re_search(self, name, root):
        if name[0] not in root.pointers:
            return False
        else:
            nextNode = root.pointers[name[0]]
            if len(name[1:]) == 0:
                if nextNode.mark == True:
                    return True, nextNode.index
                else:
                    return False, None
            else:
                return self.re_search(name[1:], nextNode)

In [3]:
def buildTree(names):
    
    tree = Tire()
    for x in range(len(names)):
        tree.insert(names[x].split(' '), x)
    
    return tree

In [4]:
def searchTerms(sentence, tree, vacabulary):
    
    sentence = sentence.replace("'", '').replace("\t", '').replace(',', '').lstrip().rstrip().lower()
    terms = sentence.split(' ')
    
    for x in range(len(terms)):  
        for y in range(x, len(terms)):
            tomatch = terms[x:y+1]
            response, index = tree.search(tomatch)
            if response:
                return vacabulary[index]
            else:
                return ''

In [6]:
import pandas as pd
import numpy as np
import collections
from collections import OrderedDict

newData = pd.read_csv("listings_1461025592.csv", warn_bad_lines=True, error_bad_lines=False)
oldData = pd.read_csv("sothebys_historical_data.csv", warn_bad_lines=True, error_bad_lines=False)

data = newData.append(oldData)
nullMaker = data[pd.isnull(data['Maker'])]

print len(makers)

record = []
clean = []
for x in makers:
    clean.append(x.replace("'", '').replace("\t", '').replace(',', '').lstrip().rstrip().lower())
    record.append(x.replace("'", '').replace("\t", '').replace(',', '').lstrip().rstrip())
    
# clean = sorted(list(set(clean)))

print len(clean), len(record)

Skipping line 65972: expected 18 fields, saw 19

/Users/gzhao/Library/Python/2.7/lib/python/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
Skipping line 225573: expected 18 fields, saw 27

/Users/gzhao/Library/Python/2.7/lib/python/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


834227
834227 834227


In [7]:
tree = buildTree(clean)
makers = []

for index, row in nullMaker.iterrows():
    terms = row['Lot Title']
    try:
        maker = searchTerms(terms, tree, record)
        makers.append(maker)
    except:
        makers.append('')

In [8]:
nullMaker['Maker'] = makers
noMaker = nullMaker[nullMaker['Maker'] == '']
len(noMaker)/float(len(data))

/Users/gzhao/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


0.6631983953806955

In [9]:
counter = collections.Counter(makers)
print counter.most_common(100)

[('', 594499), (u'three', 3233), (u'John', 2433), (u'Ruby', 1976), (u'Rolex', 1723), (u'French', 1603), (u'Robert', 1372), (u'Hermitage', 1358), (u'Barolo', 1345), (u'Charles', 1312), (u'George', 1241), (u'Pablo', 1072), (u'Montrachet', 1024), (u'Cartier', 1018), (u'Thomas', 1004), (u'Paul', 955), (u'Corton', 954), (u'James', 953), (u'Edward', 942), (u'Jean', 891), (u'David', 883), (u'Chambertin', 866), (u'Richebourg', 839), (u'group', 795), (u'German', 784), (u'Joseph', 773), (u'Echezeaux', 772), (u'Dom', 738), (u'Krug', 674), (u'Meursault', 654), (u'Dutch', 625), (u'Musigny', 625), (u'Marc', 618), (u'Breguet', 584), (u'Richard', 559), (u'Vega', 556), (u'Volnay', 504), (u'Pearl', 495), (u'Albert', 494), (u'Ornellaia', 480), (u'Sassicaia', 474), (u'Peter', 460), (u'Dominus', 457), (u'statuette', 455), (u'Alexander', 441), (u'Flemish', 437), (u'enamel', 435), (u'Piaget', 427), (u'aquamarine', 425), (u'Taylor', 419), (u'Graham', 413), (u'Swiss', 396), (u'Petit', 393), (u'Pierre', 383), (